In [1]:
import os

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Load model to test using simple message

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

d:\langchain_demo\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model = ChatGoogleGenerativeAI(model = 'gemini-pro',convert_system_message_to_human=True)

In [8]:
message = [SystemMessage("Hi you are nice bot!"),HumanMessage("Hi, How are you buddy?")]

In [9]:
model.invoke(message)

d:\langchain_demo\venv\lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


AIMessage(content='Thank you for the compliment! I am doing well, thank you for asking. How are you doing today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-42b5ae2b-5ae1-4bd7-a553-e83dc8bc51f2-0', usage_metadata={'input_tokens': 14, 'output_tokens': 22, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})

## Use output parser

In [10]:
from langchain_core.output_parsers import StrOutputParser

In [11]:
parser = StrOutputParser()

In [12]:
response = model.invoke(message)

d:\langchain_demo\venv\lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [13]:
parser.invoke(response)

'Thank you for the compliment! I am doing well, thank you for asking. How are you doing today?'

## Chaining with LCEL

In [14]:
chain = model | parser

In [15]:
chain.invoke(message)

d:\langchain_demo\venv\lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Thank you for the compliment! I am doing well, thank you for asking. How are you doing today?'

## Now lets start with prompting

In [16]:
from langchain_core.prompts import ChatPromptTemplate

In [17]:
system = "translate the following into {language}"
user = "I am hungry"

In [18]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system","translate the following into {language}"),
    ("user","{text}")]
)

In [19]:
prompt = prompt_template.invoke({"language":"french","text":"I am going to delhi for 3 day tour"})

In [20]:
model.invoke(prompt)

d:\langchain_demo\venv\lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


AIMessage(content='Je vais à Delhi pour un séjour de 3 jours', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0bede8b7-d87e-4c40-a659-f28ac66f9810-0', usage_metadata={'input_tokens': 16, 'output_tokens': 11, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

In [21]:
parser.invoke(model.invoke(prompt))

d:\langchain_demo\venv\lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Je vais à Delhi pour une visite de 3 jours'

## Chaining
### In above code we are creating prompt template separate and then invoking it. Instead we can do this in one go by doing chaining

In [22]:
chain = prompt_template | model | parser

In [23]:
chain.invoke({"language":"hindi","text":"I am learning langchain"})

d:\langchain_demo\venv\lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'मैं लैंगचैन सीख रहा हूँ'

In [24]:
chain.invoke({"language":"hindi","text":'Je vais à Delhi pour une visite de 3 jours'})

d:\langchain_demo\venv\lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'मैं दिल्ली 3 दिनों के लिए घूमने जा रहा हूँ'

## Agents in langchain
### Agents are components that enable language models to make decisions dynamically within a workflow. Rather than following a strict, pre-defined sequence of tasks, agents allow for more flexible interactions by choosing actions based on the context of the conversation or query.

In [39]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [40]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [41]:
memory = MemorySaver()

In [42]:
search = TavilySearchResults(max_results=2)

In [43]:
tools = [search]

In [44]:
agent_executor = create_react_agent(model,tools,checkpointer=memory)

In [45]:
config = {"configurable":{"thread_id":"abc123"}}

In [49]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="Hi I am shoaib!")]},config):
    print(chunk)
    print("-"*50)

d:\langchain_demo\venv\lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'agent': {'messages': [AIMessage(content='Hello Shoaib, how can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-a812621f-4271-4e10-a77b-830f7b91ecb4-0', usage_metadata={'input_tokens': 845, 'output_tokens': 11, 'total_tokens': 856, 'input_token_details': {'cache_read': 0}})]}}
--------------------------------------------------


In [50]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="I live in Lakhimpur, can u please tell me the weather of lakhimpur-kheri?")]},config):
    print(chunk)
    print("-"*50)

d:\langchain_demo\venv\lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is the weather in Lakhimpur Kheri?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-b3bea4ae-e721-4481-af95-15b7bef1b22f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is the weather in Lakhimpur Kheri?'}, 'id': 'cd8e2671-1e5c-4de2-82e4-229d225231fd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 880, 'output_tokens': 30, 'total_tokens': 910, 

d:\langchain_demo\venv\lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'agent': {'messages': [AIMessage(content='The weather in Lakhimpur Kheri is currently clear with a temperature of 21.4 degrees Celsius (70.5 degrees Fahrenheit). The humidity is 67% and the wind speed is 2.2 miles per hour (3.6 kilometers per hour).', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-97193462-dc54-47b0-a9f3-992d4188b33a-0', usage_metadata={'input_tokens': 1639, 'output_tokens': 57, 'total_tokens': 1696, 'input_token_details': {'cache_read': 0}})]}}
--------------------------------------------------
